In [1]:
import os
import time
import zipfile
import pickle
import numpy as np
import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim, SparseRandomProjection
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def computeResults(testLabels, predictLabels):
    precision = precision_score(testLabels, predictLabels, zero_division=0)
    recall = recall_score(testLabels, predictLabels, zero_division=0)
    accuracy = accuracy_score(testLabels, predictLabels)
    return precision, recall, accuracy

###############################################################################
# FLAST functions

def flastVectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

def flastXGBClassification(trainData, trainLabels, testData, eta, max_depth, params):
    t0 = time.perf_counter()
    xgb_model = xgb.XGBClassifier(eta=eta, max_depth=max_depth, use_label_encoder=False, eval_metric="logloss")
    xgb_model.fit(trainData, trainLabels)
    trainTime = time.perf_counter() - t0

    t0 = time.perf_counter()
    predictLabels = xgb_model.predict(testData)
    testTime = time.perf_counter() - t0

    return trainTime, testTime, predictLabels

###############################################################################
# FLAST XGB analysis

def flastXGB(outDir, flakyZip, nonFlakyZip, extractDir, kf, dim, eps, eta, max_depth, params):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    if len(dataPoints) > 0:
        print(f"Sample document: {dataPoints[0]}")

    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataPointsList = np.array([Z[i].toarray() for i in range(Z.shape[0])])
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Storage calculation
    xgb_data = (dataPointsList, dataLabelsList)
    pickleDumpXGB = os.path.join(outDir, f"flast-xgb-eta{eta}-depth{max_depth}.pickle")
    with open(pickleDumpXGB, "wb") as pickleFile:
        pickle.dump(xgb_data, pickleFile)
    storage = os.path.getsize(pickleDumpXGB)
    os.remove(pickleDumpXGB)

    avgP, avgR, avgA = 0, 0, 0
    avgTPrep, avgTPred = 0, 0
    avgFlakyTrain, avgNonFlakyTrain, avgFlakyTest, avgNonFlakyTest = 0, 0, 0, 0
    successFold, precisionFold = 0, 0

    for trnIdx, tstIdx in kf.split(dataPointsList, dataLabelsList):
        trainData, testData = dataPointsList[trnIdx], dataPointsList[tstIdx]
        trainLabels, testLabels = dataLabelsList[trnIdx], dataLabelsList[tstIdx]
        if sum(trainLabels) == 0 or sum(testLabels) == 0:
            print("Skipping fold with no flaky/non-flaky examples...")
            continue

        successFold += 1
        avgFlakyTrain += sum(trainLabels)
        avgNonFlakyTrain += len(trainLabels) - sum(trainLabels)
        avgFlakyTest += sum(testLabels)
        avgNonFlakyTest += len(testLabels) - sum(testLabels)

        trainData = trainData.reshape((trainData.shape[0], -1))
        testData = testData.reshape((testData.shape[0], -1))

        trainTime, testTime, predictLabels = flastXGBClassification(trainData, trainLabels, testData, eta, max_depth, params)
        preparationTime = (vecTime * len(trainData) / len(dataPoints)) + trainTime
        predictionTime = (vecTime / len(dataPoints)) + (testTime / len(testData))
        precision, recall, accuracy = computeResults(testLabels, predictLabels)

        print(f"Precision: {precision}, Recall: {recall}, Accuracy: {accuracy}")
        if precision != "-":
            precisionFold += 1
            avgP += precision
            avgA += accuracy
        avgR += recall
        avgTPrep += preparationTime
        avgTPred += predictionTime

    if precisionFold > 0:
        avgP /= precisionFold
        avgA /= precisionFold
    else:
        avgP = avgA = "-"
    avgR /= successFold
    avgTPrep /= successFold
    avgTPred /= successFold
    avgFlakyTrain /= successFold
    avgNonFlakyTrain /= successFold
    avgFlakyTest /= successFold
    avgNonFlakyTest /= successFold

    return avgFlakyTrain, avgNonFlakyTrain, avgFlakyTest, avgNonFlakyTest, avgP, avgR, avgA, storage, avgTPrep, avgTPred

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "C:/Users/kdeep/Downloads/Flakiness ML/cleaned_flaky_files.zip"
    nonFlakyZip = "C:/Users/kdeep/Downloads/Flakiness ML/reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results/"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    numSplit = 10
    kf = StratifiedKFold(n_splits=numSplit)

    outFile = "params-xgb.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("eta,max_depth,dim,eps,avgFlakyTrain,avgNonFlakyTrain,avgFlakyTest,avgNonFlakyTest,precision,recall,accuracy,storage,preparationTime,predictionTime\n")

    # XGB parameters to vary
    eta_values = [0.1, 0.3, 0.5]  # Learning rate
    depth_values = [3, 5, 7]  # Tree depth
    dim_values = [0, 100, 200]  # Dimensionality reduction
    eps_values = [0.1, 0.3, 0.5]  # JL epsilon

    for eta in eta_values:
        for depth in depth_values:
            for dim in dim_values:
                for eps in eps_values:
                    print(f"eta={eta}, depth={depth}, dim={dim}, eps={eps}")
                    results = flastXGB(outDir, flakyZip, nonFlakyZip, extractDir, kf, dim, eps, eta, depth, {})
                    print(f"Results to be written: eta={eta}, depth={depth}, dim={dim}, eps={eps}, {results}")
                    with open(os.path.join(outDir, outFile), "a") as fo:
                        fo.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(eta, depth, dim, eps, *results))

    print("XGBoost analysis completed. Results saved to:", outFile)
    

ModuleNotFoundError: No module named 'xgboost'